# 随机数种子
随机种子（Random Seed）是用于初始化随机数生成器的一个固定值。通过设置相同的随机种子，随机数生成器会产生相同的随机数序列，这对于实验和结果的可重复性至关重要。
常见的随机操作包括：
+ 数据集的随机拆分(如训练集和测试集的划分)
+ 模型参数的随机初始化
+ 数据增强中的随机变换

In [ ]:
# example of setting random seed for reproducibility
# the results will be the same each time
import numpy as np
import random
import torch
seed = 66
np.random.seed(seed)
random.seed(seed)
# torch.manual_seed(seed)
random_matrix_first = np.random.randn(3, 3)
print(random_matrix_first)
random_data_one_first = random.random()
random_data_two_first = random.random()
print(random_data_one_first, random_data_two_first)


In [ ]:
# example of not setting random seed
# the results will be different each time
import numpy as np
import random

random_matrix_second = np.random.randn(3, 3)
print(random_matrix_second)
random_data_one_second = random.random()
random_data_two_second = random.random()
print(random_data_one_second, random_data_two_second)

# argparser
是python标准库中的一个模块，用于解析命令行参数。argparser模块使得编写用户友好的命令行界面变得容易。它可以解析命令行参数并生成帮助信息。argparser模块还可以自动生成帮助信息。

In [ ]:
# example
import argparse
# create the object of ArgumentParser
parser = argparse.ArgumentParser(description = "Description of the program")
# add_argument() method is used to add the arguments
parser.add_argument("--arg1", help = "Description of the arg1")
parser.add_argument("--optional_arg", default = "default_value", help = "Description of the optional_arg")
# parse_args() method is used to parse the arguments
args = parser.parse_args()
# use the arguments
print(args.arg1)
print(args.optional_arg)

# function "DataLoader" in torch.utils.data

## The parameter "collate_fn" of function "DataLoader" in torch.utils.data
collate_fn：将一个batch的数据样本组织成一个batch，这个batch中的每个数据样本的形状可能不一样，所以需要用一个函数来组织这个batch。默认的collate_fn函数会将数据样本组织成一个list，这样就不会出现形状不一样的问题。如果数据样本的形状一样，可以不用设置collate_fn参数。

作用：
+ 自定义合并逻辑：当数据集中的样本结构复杂或者每个样本的形状不一致时，通过"collate_fn"自定义合并逻辑能够更灵活地处理这些样本。
+ 处理变长序列：对于自然语言处理等任务，样本（例如句子）长度不一，需要自定义合并逻辑进行填充和打包。
+ 处理多模态数据：当一个样本包含多种不同模态的数据（如图像和文本）时，需要自定义合并逻辑以正确处理和打包这些数据。

使用方法：
+ 定义一个函数，该函数接受一个batch的数据样本列表，返回一个batch的数据样本。'collate_fn'函数中的'batch'是一个列表，包含了多个通过'\_\_getitem_\_'方法获取的数据样本。
+ 将该函数传递给DataLoader的collate_fn参数。

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

data = [
    {'input': torch.tensor([1, 2, 3]), 'label': torch.tensor(0)},
    {'input': torch.tensor([4, 5]), 'label': torch.tensor(1)},
    {'input': torch.tensor([6, 7, 8, 9]), 'label': torch.tensor(2)}
]

dataset = CustomDataset(data)

def custom_collate_fn(batch):
    inputs = [item['input'] for item in batch]
    labels = [item['label'] for item in batch]

    lengths = [len(item) for item in inputs]
    max_length = max(lengths)

    padded_inputs = torch.zeros((len(inputs), max_length))
    for i, input in enumerate(inputs):
        padded_inputs[i, :len(input)] = input
    
    labels = torch.stack(labels)

    return padded_inputs, labels, lengths
"""
first batch:
[
    {'input': torch.tensor([1, 2, 3]), 'label': torch.tensor(0)},
    {'input': torch.tensor([4, 5]), 'label': torch.tensor(1)}
]

second batch:
[
    {'input': torch.tensor([6, 7, 8, 9]), 'label': torch.tensor(2)}   
]
"""
dataloader = DataLoader(dataset, batch_size = 2, shuffle = False, collate_fn = custom_collate_fn)

for batch in dataloader:
    padded_inputs, labels, lengths = batch
    print("Padded inputs:\n", padded_inputs)
    print("Labels:\n", labels)
    print("Lengths:\n", lengths)
    print("\n")

## The parameter "sampler" and "batch_sampler" of function "DataLoader" in torch.utils.data
sampler: 

定义如何从数据集中采样单个样本。它接受一个"sampler"对象。pytorch中提供了一些内置的采样器:torch.utils.data.RandomSampler, torch.utils.data.SequentialSampler, torch.utils.data.SubsetRandomSampler, torch.utils.data.WeightedRandomSampler等。这些采样器，都接收一个dataset对象，返回一个索引序列，用于获取数据集中的样本。

batch_sampler: 

将单个样本采样器生成的样本索引打包成批次。它接受一个"batch_sampler"对象。pytorch中提供了一些内置的batch采样器:torch.utils.data.BatchSampler, torch.utils.data.WeightedRandomSampler等。这些batch采样器，接收一个sampler对象，返回一个索引序列，用于获取数据集中的样本。

通过这两个参数，pytorch的'DataLoader'提供了灵活且强大的数据加载和采样机制，满足不同的实验和训练要求。另外可以，**自定义'sampler'和'batch_sampler'可以满足特定的数据采样需求**。

自定义'sampler'：
+ 继承torch.utils.data.Sampler类，实现\_\_iter\_\_和\_\_len\_\_方法。

自定义'batch_sampler'：
+ 继承torch.utils.data.BatchSampler类，实现\_\_iter\_\_和\_\_len\_\_方法。

In [ ]:
# example: sampler
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset

data = torch.arange(10).view(-1, 1)
dataset = TensorDataset(data)

print("Example of sampler")
random_sampler = torch.utils.data.RandomSampler(dataset)
dataloader_one = DataLoader(dataset, sampler = random_sampler, batch_size = 2)
for batch in dataloader_one:
    print(batch)

print('-------------------')

# example: batch_sampler
print("Example of batch sampler")
batch_sampler = torch.utils.data.BatchSampler(random_sampler, batch_size = 3, drop_last = False)
dataloader_two = DataLoader(dataset, batch_sampler = batch_sampler)
for batch in dataloader_two:
    print(batch)

print('-------------------')

# example: customize sampler
class ReverseSampler(torch.utils.data.Sampler):
    def __init__(self, data_source):
        self.data_source = data_source
    
    def __iter__(self):
        return iter(range(len(self.data_source) - 1, -1, -1))
    
    def __len__(self):
        return len(self.data_source)

print("Exampler of customized sampler")
reverse_sampler = ReverseSampler(dataset)
dataloader_three = DataLoader(dataset, sampler = reverse_sampler, batch_size = 2) 

for batch in dataloader_three:
    print(batch)

print('-------------------')

# example: customize batch sampler
class CustomBatchSampler(torch.utils.data.BatchSampler):
    def __init__(self, sampler, batch_size, drop_last):
        self.sampler = sampler
        self.batch_size = batch_size
        self.drop_last = drop_last
    
    def __iter__(self):
        batch = []
        for index in self.sampler:
            batch.append(index)
            if len(batch) == self.batch_size:
                yield batch
                batch = []
        
        if len(batch) > 0 and not self.drop_last:
            yield batch
    
    def __len__(self):
        if self.drop_last:
            return len(self.sampler) // self.batch_size
        else: 
            return (len(self.sampler) + self.batch_size - 1) // self.batch_size

batch_sampler = CustomBatchSampler(reverse_sampler, batch_size = 3, drop_last = False)
dataloader_four = DataLoader(dataset, batch_sampler = batch_sampler)

for batch in dataloader_four:
    print(batch)

# Xavier 初始化
Xavier初始化是用于初始化神经网络权重的一种方法，其目的是让信号能够更好地在网络层间传播，从而避免梯度消失或者爆炸的问题。

Xavier初始化分为均匀分布(Xavier Uniform)和正态分布(Xavier Normal)两种方式。

**Xavier Uniform**:

W ~ U($-\sqrt{\frac{6}{n_{in}+n_{out}}}, \sqrt{\frac{6}{n_{in}+n_{out}}}$)
+ U(a, b)表示均匀分布，a和b分别是分布的上下界。
+ $n_{in}$是输入单元的数量，$n_{out}$是输出单元的数量。

**Xavier Normal**:

W ~ N(0, $\sqrt{\frac{2}{n_{in}+n_{out}}}$)
+ N($\mu$, $\sigma^2$)表示正态分布，$\mu$和$\sigma^2$分别是分布的均值和方差。
+ $n_{in}$是输入单元的数量，$n_{out}$是输出单元的数量。

总结：
+ Xavier Uniform：权重从均匀分布中采样，适用于浅层网络，能够提供平滑的信号传播
+ Xavier Normal： 权重从正态分布中采样，适用于深层网络，能防止梯度消失或爆炸

In [ ]:
import torch
import torch.nn as nn
seed = 42
torch.manual_seed(seed)
# Example: Xavier Uniform Initialization
linear = nn.Linear(3, 2)
nn.init.xavier_uniform_(linear.weight).requires_grad_(False)
print(linear.weight)

# Example: Xavier Normal Initialization
linear = nn.Linear(3, 2)
nn.init.xavier_normal_(linear.weight).requires_grad_(False)
print(linear.weight)


# torch.autograd
https://pytorch.org/docs/stable/autograd.html
## 自定义的前向和反向向传播函数
方法：
+ 继承torch.autograd.Function类
+ 实现forward和backward方法

In [ ]:
import torch
from torch.autograd import Function

class MyFunction(Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)        # 上下文对象，用于在前向传播中保存信息，以便在后向传播中使用
        return input * 2
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors          # 从上下文对象中获取保存的信息
        print("ctx saved tensor: ", ctx.saved_tensors)
        print("grad output: ", grad_output)
        grad_input = grad_output * 2        # grad_output是output.sum()的梯度，即一个标量1。根据链式法则，input的梯度是grad_output * 2
        print("gard input: ", grad_input)
        return grad_input

input = torch.tensor([1.0, 2.0, 3.0], requires_grad = True)
output = MyFunction.apply(input)            # apply()方法用于调用自定义的Function; 类中定义的静态方法不需要实例化即可调用
print("output: ", output)
output.sum().backward()                     # 从标量"output_sum"出发，计算计算图反向传播，计算每个叶子节点的梯度

print("input.grad: ", input.grad)

a = torch.tensor(1.0, requires_grad = True)
b = torch.tensor(2.0, requires_grad = True)
c = torch.tensor(3.0, requires_grad = True)
f1 = a + b
f2 = b + c
output = f1 * f2
output.backward()
print(a.grad, b.grad, c.grad)

## @once_differentiable
@once_differentiable 是pytorch中的一个装饰器，主要用于装饰自定义的自动求导函数。这个装饰器主要用于确保编写的自定义函数是可微的(differentiable)，并且仅在一次微分时有效。这是为了增强梯度计算的稳定性和准确性。

具体来说，'@once_differentiable"用于装饰那些实现了"torch.autograd.Function"类的自定义函数。这些自定义函数需要实现"forward"和"backward"方法，其中"forward"方法定义了前向传播和计算，"backward"方法定义了反向传播时的梯度计算。

使用这个装饰器，可以确保自定义函数在反向传播时能够正确地计算梯度，避免由于函数不可微导致的梯度计算问题。

In [2]:
import torch
from torch.autograd import Function
from torch.autograd.function import once_differentiable

class MyFunction(Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input ** 2

    @staticmethod
    @once_differentiable   
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        # 返回输入的梯度
        grad_input = 2 * input * grad_output
        return grad_input

input = torch.tensor([1.0, 2.0, 3.0], dtype = torch.float32, requires_grad = True)
output = MyFunction.apply(input)
output.sum().backward()
print("input.grad: ", input.grad)


input.grad:  tensor([2., 4., 6.])


# data create & process in pytorch

## 维度转换

### torch.flatten()

"flatten"方法用于将张量的多个维度展平成一个维度。这个方法特别适用于将高维数据转换为一维数据。

调用方法：torch.flatten(input, start_dim=0, end_dim=-1) -> Tensor

In [7]:
import torch

data = torch.randn((3, 4, 5, 2, 9))
print("before: ", data.shape)

data_flatten = data.flatten(start_dim = 2, end_dim = 3)
print("after: ", data_flatten.shape)

before:  torch.Size([3, 4, 5, 2, 9])
after:  torch.Size([3, 4, 10, 9])


### torch.transpose()
"transpose"方法用于交换张量的两个维度。它只能同时交换两个维度，而不能处理超过两个维度的交换。

In [3]:
import torch

data = torch.randn((3, 4, 8, 9))
print("before: ", data.shape)

data_transpose = data.transpose(0, 2)       # the same as data.transpose(2, 0)
print("after: ", data_transpose.shape)

before:  torch.Size([3, 4, 8, 9])
after:  torch.Size([8, 4, 3, 9])


### torch.permute()
"permute"方法用于根据指定的新维度顺序重排张量的所有维度。它允许任意数量的维度重排。

In [1]:
import torch

data = torch.randn((3, 2, 4, 5))
print("before: ", data.shape)

data_permute = data.permute(3, 1, 0, 2)
print("after: ", data_permute.shape)

before:  torch.Size([3, 2, 4, 5])
after:  torch.Size([5, 2, 3, 4])


### torch.view()
"torch.view"方法用于在不改变数据内容的前提下，重新调整张量的形状。"view"是基于原始张量创建一个新张量，这两个张量共享相同的数据存储，因此，修改其中一个张量的内容会影响到另一个张量

In [6]:
import torch

data = torch.zeros((3, 4, 5))
print("before: ", data.shape)

data_view = data.view(3, -1)
print("after: ", data_view.shape)

data_view[0, 0] = 1     # 同时改变了data的值
print(data)

before:  torch.Size([3, 4, 5])
after:  torch.Size([3, 20])
tensor([[[1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


## 维度拼接

### torch.cat()
"cat"方法用于将一组张量沿指定维度拼接起来。torch.cat(tensors, dim=0) -> Tensor
+ 被拼接的张量必须在除了指定维度外的所有维度形状相同
+ 拼接后的张量在指定维度上的大小是所有输入张量在该维度上的大小之和

In [9]:
import torch

data_one = torch.randn((3, 20, 4))
data_two = torch.randn((3, 60, 4))

data_cat_first = torch.cat([data_one, data_two], dim = 1)
print("after: ", data_cat_first.shape)

data_three = torch.randn((10, 5, 9))
data_four = torch.randn((10, 5, 100))
data_cat_second = torch.cat([data_three, data_four], dim = 2)
print("after: ", data_cat_second.shape)


after:  torch.Size([3, 80, 4])
after:  torch.Size([10, 5, 109])


### torch.stack()
"stack"方法用于将一组张量沿新维度拼接起来。torch.stack(tensors, dim=0) -> Tensor
+ 被拼接的张量必须在所有维度上形状相同
+ 堆叠后的张量会增加一个新维度，这个新维度的大小等于输入张量的数量

In [53]:
import torch

data_one = torch.randn((3, 4))
data_two = torch.randn((3, 4))
data_three = torch.randn((3, 4))
data_four = torch.randn((3, 4))
data_five = torch.randn((3, 4))

data_stack = torch.stack([data_one, data_two, data_three, data_four, data_five], dim = 2)      # 参数dim的值不能超过堆叠后的数据维度，例如堆叠后的维度数为3， 那么dim的值不能超过2
print("after: ", data_stack.shape)
print(data_stack)

after:  torch.Size([3, 4, 5])
tensor([[[ 0.5067, -1.2020,  1.8099, -0.4309,  0.0798],
         [ 0.6174,  0.8297,  0.8262, -0.4378,  1.4239],
         [ 0.6762,  1.5456,  1.0412,  2.4749,  0.0731],
         [ 1.6772, -0.2456, -1.4661,  0.7084, -0.6128]],

        [[ 1.2085,  0.4101,  0.0168, -0.0517,  1.0677],
         [-1.6646,  1.4566,  1.0326,  0.6308, -1.3728],
         [-0.2930, -0.4203,  1.2512, -0.1092,  0.1509],
         [ 0.6968,  0.4492, -0.8958, -0.5760, -0.4606]],

        [[ 1.4241,  0.6967, -1.1816, -0.4726,  1.2660],
         [-0.0825, -0.2198, -1.4862, -0.9830, -0.0110],
         [ 0.4215,  0.8529, -0.0222,  0.1961,  0.3469],
         [-0.3362,  0.0602, -1.3227, -0.9573, -0.3047]]])


## 按维度计算

### torch.prod()

"prod"方法用于计算张量的所有元素或指定维度上的元素的乘积。这个方法在处理需要计算累积乘积的任务时非常重要。

调用方法：torch.prod(input, dim = None, keepdim = False, dype = None) -> Tensor; 若未指定dim，则计算所有元素的乘积。

In [33]:
import torch

data = torch.tensor([
        [[2, 3, 4], [5, 6, 7]],
        [[1, 2, 3], [4, 5, 6]]
    ])
print(data)
print(data.shape)
print(data.prod(1))
print(data.prod(1).shape)
print(data.prod(1, keepdim = True))
print(data.prod(1, keepdim = True).shape)

tensor([[[2, 3, 4],
         [5, 6, 7]],

        [[1, 2, 3],
         [4, 5, 6]]])
torch.Size([2, 2, 3])
tensor([[10, 18, 28],
        [ 4, 10, 18]])
torch.Size([2, 3])
tensor([[[10, 18, 28]],

        [[ 4, 10, 18]]])
torch.Size([2, 1, 3])


### torch.cumsum()

"cumsum"方法用于计算张量沿指定维度的累计和(cumulative sum)。该方法返回一个张量，其元素是沿给定维度的累积和。**累积和的结果会与原始张量具有相同形状**。

调用方法：torch.cumsum(input, dim, dtype = None) -> Tensor

In [38]:
import torch

data = torch.tensor([
        [[2, 3, 4], [5, 6, 7]],
        [[1, 2, 3], [4, 5, 6]]
    ])
print("before data: \n", data)
print("before data.shape: \n", data.shape)

data_cumsum_dim1 = data.cumsum(dim = 1)
data_cumsum_dim2 = data.cumsum(dim = 2)
print("after data_cumsum_dim1: \n", data_cumsum_dim1)
print("after data_cumsum_dim1.shape: \n", data_cumsum_dim1.shape)
print("after data_cumsum_dim2: \n", data_cumsum_dim2)
print("after data_cumsum_dim2.shape: \n", data_cumsum_dim2.shape)

before data: 
 tensor([[[2, 3, 4],
         [5, 6, 7]],

        [[1, 2, 3],
         [4, 5, 6]]])
before data.shape: 
 torch.Size([2, 2, 3])
after data_cumsum_dim1: 
 tensor([[[ 2,  3,  4],
         [ 7,  9, 11]],

        [[ 1,  2,  3],
         [ 5,  7,  9]]])
after data_cumsum_dim1.shape: 
 torch.Size([2, 2, 3])
after data_cumsum_dim2: 
 tensor([[[ 2,  5,  9],
         [ 5, 11, 18]],

        [[ 1,  3,  6],
         [ 4,  9, 15]]])
after data_cumsum_dim2.shape: 
 torch.Size([2, 2, 3])


## 转换为张量

### torch.tensor()
将数据转换为张量。torch.tensor(data, dtype=None, device=None, requires_grad=False) -> Tensor

特点：**总是会创建数据的副本**，即使输入数据已经是一个张量或者是一个numpy数组。

In [25]:
import torch
import numpy as np
data_list = [1, 2, 3, 4]
data_numpy = np.array(data_list)
data_tensor = torch.tensor(data_numpy)
print("list: ", data_list)
print("numpy: ", data_numpy)
print("tensor: ", data_tensor)
print(data_numpy.ctypes.data == data_tensor.data_ptr())

list:  [1, 2, 3, 4]
numpy:  [1 2 3 4]
tensor:  tensor([1, 2, 3, 4])
False


### torch.as_tensor()
将数据转化成张量。如果输入数据已经是一个张量或者是一个numpy数组，则可以共享内存，不会进行数据复制。**它会尽量与输入数据共享内存**。

In [26]:
import torch
import numpy as np

data_list = [1, 2, 3, 4]
data_numpy = np.array(data_list)
data_as_tensor = torch.as_tensor(data_numpy)
print("list: ", data_list)
print("numpy: ", data_numpy)
print("tensor: ", data_as_tensor)
print(data_numpy.ctypes.data == data_as_tensor.data_ptr())

list:  [1, 2, 3, 4]
numpy:  [1 2 3 4]
tensor:  tensor([1, 2, 3, 4])
True


## 掩码

### tensor.masked_fill()
"mask_fill"方法用于根据掩码(mask)将张量中指定位置的元素替换为一个指定的值。掩码通常是一个布尔张量或与原张量形状相同的二进制张量，其中'True'或'1'的位置表示需要替换的位置。

调用方法: tensor.masked_fill(mask, value) -> Tensor
+ mask: 一个布尔张量或与原张量形状相同的二进制张量。'mask'中为'True'或'1'的位置会被'value'替换
+ value:用于替换掩码中指定位置的值

输出结果与原张量具有相同的形状。

In [3]:
import torch

data = torch.randn((2, 6, 2))
print("data: ", data)
print("data.shape: ", data.shape)
mask = torch.tensor([
    [True, False, True, False, True, False],
    [False, True, False, True, False, True]
])
print("mask's shape: ", mask[..., None].shape)          # mask[..., None] is equivalent to mask[:, :, None]
data_masked = data.masked_fill(mask[..., None], 0)
print("data_masked: ", data_masked)
print("data_masked.shape: ", data_masked.shape)



data:  tensor([[[ 1.2971, -0.1103],
         [ 0.5934,  0.9164],
         [ 0.3821,  0.7289],
         [ 0.0761, -1.7662],
         [-0.1889,  1.0144],
         [ 0.2463, -0.9795]],

        [[-1.1312, -1.4169],
         [ 0.2797, -0.3052],
         [-1.1987, -0.4208],
         [ 0.4150, -0.2298],
         [ 0.4249,  1.3761],
         [-0.3598, -0.2692]]])
data.shape:  torch.Size([2, 6, 2])
mask's shape:  torch.Size([2, 6, 1])
data_masked:  tensor([[[ 0.0000,  0.0000],
         [ 0.5934,  0.9164],
         [ 0.0000,  0.0000],
         [ 0.0761, -1.7662],
         [ 0.0000,  0.0000],
         [ 0.2463, -0.9795]],

        [[-1.1312, -1.4169],
         [ 0.0000,  0.0000],
         [-1.1987, -0.4208],
         [ 0.0000,  0.0000],
         [ 0.4249,  1.3761],
         [ 0.0000,  0.0000]]])
data_masked.shape:  torch.Size([2, 6, 2])


## 建立特殊值张量

### torch.zeros()

"zeros"方法用于创建一个全为0的张量。torch.zeros(*size, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) -> Tensor

In [39]:
import torch

size = (3, 4)
data = torch.zeros(size, dtype = torch.float32)
print(data)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


### tensor.new_zeros()

"tensor.new_zeros"方法用于创建一个与给定张量具有相同数据类型和设备的全零张量。

特点：
+ 新创建的张量继承了调用张量的dtype 和 device属性
+ 不需要显式指定dtype 和 device，除非需要覆盖调用张量的dtype 和 device属性

调用方法：tensor.new_zeros(size, dtype = None, device = None, requires_grad = False) -> Tensor

In [43]:
import torch

data = torch.randn((3, 4), dtype = torch.float32, device = torch.device('cpu'))

data_as_zeros = data.new_zeros((2, 5))
print(data_as_zeros.shape, data_as_zeros.dtype, data_as_zeros.device)
print(data_as_zeros)

torch.Size([2, 5]) torch.float32 cpu
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


### torch.zeros_like()

"torch.zeros_like"方法用于创建一个与给定张量具有相同形状的全为0的张量。

调用方法：torch.zeros_like(input, dtype=None, layout=None, device=None, requires_grad=False) -> Tensor

与"tensor.new_zeros"方法的区别：
+ "tensor.new_zeros"方法创建的张量与调用张量具有相同的dtype和device属性, 尽管可以覆盖这些属性。但是需要显式给出形状(size)参数
+ "torch.zeros_like"方法创建的张量与给定张量具有相同的形状, 不需要显式给出形状参数

In [45]:
import torch

data = torch.zeros((3, 4), dtype = torch.float32)
data_zeros_like = torch.zeros_like(data)
print("data: \n", data)
print("data_zeros_like: \n", data_zeros_like)

data: 
 tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
data_zeros_like: 
 tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
